In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [21]:
data = pd.read_csv('SPData.csv')

# Convert Sector into numerical
data['Sector'] = data['Sector'].astype('category')
data['Sector'] = data['Sector'].cat.codes

data.dropna(subset=['Result'], inplace=True)

bins = [-5, 0, .25, .5, 10]
data['Result_Bin'] = pd.cut(data['Result'], bins).astype('category').cat.codes

pct_change = data['Result']
data = data.drop(['Result'], axis=1)


display(data)

,Ticker,year,Sector,CurrentRatio,QuickRatio,CashRatio,OperatingCashFlowRatio,equityRatio,DebtToEquity,DebtToAssets,...,AssetTurnover,ReceivablesTurnover,GrossMargin,GrossProfitMargin,OperatingMargin,ReturnOnEquity,ReturnOnAssets,EPS,PE,Result_Bin
0,MMM,2019,7,1.406528,1.854804,0.255151,0.766645,0.225330,3.431680,0.773260,...,0.719586,6.475116,0.466766,0.142208,0.190690,0.454139,0.102331,7.920277,15.462843,0
1,MMM,2020,7,1.885003,2.418344,0.583040,1.020760,0.271777,2.674516,0.726871,...,0.679790,6.663354,0.484060,0.167288,0.211969,0.418435,0.113721,9.321330,19.056295,0
2,MMM,2021,7,1.704815,2.256558,0.505147,0.825014,0.319638,2.123820,0.678854,...,0.751083,7.411950,0.468392,0.167473,0.216829,0.393527,0.125786,10.226252,17.092283,1
3,MMM,2022,7,1.542371,2.106479,0.383808,0.587105,0.316909,2.152221,0.682058,...,0.736821,7.384898,0.438137,0.168775,0.126618,0.392406,0.124357,10.206714,17.284701,0
4,AOS,2019,7,1.957469,2.352772,0.487932,0.595173,0.545062,0.834653,0.454938,...,0.978646,5.076675,0.394527,0.123634,0.155412,0.221982,0.120994,2.278726,25.856549,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,ZION,2022,5,0.023772,0.023772,0.023772,0.017499,0.054643,17.300634,0.945357,...,0.035200,8.165803,-0.290609,0.287754,0.306789,0.185367,0.010129,NaN,NaN,0
1956,ZTS,2019,6,2.629014,3.409745,1.069767,0.993909,0.234560,3.263294,0.765440,...,0.542226,5.764273,0.681789,0.239617,0.322364,0.553914,0.129926,3.137419,46.805992,1
1957,ZTS,2020,6,3.046544,3.796774,1.659908,0.979724,0.276949,2.609711,0.722757,...,0.490484,6.589339,0.691835,0.245393,0.339925,0.434598,0.120362,3.444780,70.840508,0
1958,ZTS,2021,6,3.856427,4.926544,1.937674,1.231497,0.326835,2.059432,0.673094,...,0.559424,6.863195,0.703832,0.261960,0.360468,0.448382,0.146547,4.294750,38.535420,2


In [27]:
# Split data into training and testing
X = data[data.columns[2:-1]]
y = data['Result_Bin']

train_idx = data[data['year'] < 2022].index.values
test_idx = data[data['year'] == 2022].index.values

X_train = X.loc[train_idx]
y_train = y.loc[train_idx]
X_test = X.loc[test_idx]
y_test = y.loc[test_idx]

IQR = X_train.quantile(0.75) - X_train.quantile(0.25)
lower_bound = X_train.quantile(0.25) - (1.5 * IQR)
upper_bound = X_train.quantile(0.75) + (1.5 * IQR)

for i in X_train.columns:
    X_train.loc[X_train[i] < lower_bound[i], i ] = lower_bound[i]
    X_train.loc[X_train[i] > upper_bound[i], i ] = upper_bound[i]
    X_test.loc[X_test[i] < lower_bound[i], i ] = lower_bound[i]
    X_test.loc[X_test[i] > upper_bound[i], i ] = upper_bound[i]

    X_test[i].fillna(X_train[i].median(), inplace=True)
    X_train[i].fillna(X_train[i].median(), inplace=True)

    # Normalize data
    X_train[i] = (X_train[i] - X_train[i].mean()) / X_train[i].std()
    X_test[i] = (X_test[i] - X_train[i].mean()) / X_train[i].std()

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
display(X_train)




(1470, 18) (1470,) (490, 18) (490,)


,Sector,CurrentRatio,QuickRatio,CashRatio,OperatingCashFlowRatio,equityRatio,DebtToEquity,DebtToAssets,InterestCoverage,AssetTurnover,ReceivablesTurnover,GrossMargin,GrossProfitMargin,OperatingMargin,ReturnOnEquity,ReturnOnAssets,EPS,PE
0,0.520270,-0.078323,0.090826,-0.465887,0.271123,-0.522862,0.498393,0.564534,0.384647,0.367780,-0.251601,0.309165,0.087515,0.126083,1.533784,0.604722,0.744950,-0.503989
1,0.520270,0.447057,0.595732,0.260708,0.774059,-0.300424,0.162812,0.342214,0.300816,0.267909,-0.209079,0.377640,0.303528,0.310656,1.341777,0.782801,1.063277,-0.362110
2,0.520270,0.249205,0.450779,0.088099,0.386645,-0.071213,-0.081260,0.112083,0.602163,0.446826,-0.039974,0.315602,0.305119,0.352817,1.207827,0.971439,1.268881,-0.439654
4,0.520270,0.526628,0.536983,0.049952,-0.068248,1.008360,-0.652627,-0.961057,2.251846,1.017918,-0.567504,0.023140,-0.072462,-0.179918,0.285307,0.896518,-0.536841,-0.093618
5,0.520270,0.381795,0.368000,0.401602,0.009012,1.198550,-0.707849,-1.151388,2.251846,0.860779,-0.596293,-0.023455,-0.111307,-0.186098,0.095052,0.710889,-0.570168,0.475327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1953,-0.182991,-1.598164,-1.550943,-0.981708,-1.226500,-1.138469,1.911915,1.187356,-0.714214,-1.352160,0.255227,-2.547952,0.526620,0.088701,-0.540892,-0.891786,-0.306650,-0.356961
1954,-0.182991,-1.481763,-1.455965,-0.746796,-1.231500,-1.218497,1.911915,1.267443,1.437699,-1.360032,0.288365,-2.398021,2.216956,2.379923,-0.094912,-0.805817,0.549511,-0.871570
1956,0.168639,1.264006,1.483981,1.339287,0.720917,-0.478656,0.423763,0.527056,-0.111271,-0.077323,-0.412178,1.160525,0.926492,1.268221,2.031374,1.036173,-0.341741,0.733521
1957,0.168639,1.722466,1.830741,2.269447,0.692841,-0.275653,0.134090,0.322495,-0.028442,-0.207174,-0.225799,1.200302,0.976246,1.420543,1.428699,0.886628,-0.271907,1.682467


In [28]:
# show where nan in X_train
X_train


,Sector,CurrentRatio,QuickRatio,CashRatio,OperatingCashFlowRatio,equityRatio,DebtToEquity,DebtToAssets,InterestCoverage,AssetTurnover,ReceivablesTurnover,GrossMargin,GrossProfitMargin,OperatingMargin,ReturnOnEquity,ReturnOnAssets,EPS,PE
0,0.520270,-0.078323,0.090826,-0.465887,0.271123,-0.522862,0.498393,0.564534,0.384647,0.367780,-0.251601,0.309165,0.087515,0.126083,1.533784,0.604722,0.744950,-0.503989
1,0.520270,0.447057,0.595732,0.260708,0.774059,-0.300424,0.162812,0.342214,0.300816,0.267909,-0.209079,0.377640,0.303528,0.310656,1.341777,0.782801,1.063277,-0.362110
2,0.520270,0.249205,0.450779,0.088099,0.386645,-0.071213,-0.081260,0.112083,0.602163,0.446826,-0.039974,0.315602,0.305119,0.352817,1.207827,0.971439,1.268881,-0.439654
4,0.520270,0.526628,0.536983,0.049952,-0.068248,1.008360,-0.652627,-0.961057,2.251846,1.017918,-0.567504,0.023140,-0.072462,-0.179918,0.285307,0.896518,-0.536841,-0.093618
5,0.520270,0.381795,0.368000,0.401602,0.009012,1.198550,-0.707849,-1.151388,2.251846,0.860779,-0.596293,-0.023455,-0.111307,-0.186098,0.095052,0.710889,-0.570168,0.475327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1953,-0.182991,-1.598164,-1.550943,-0.981708,-1.226500,-1.138469,1.911915,1.187356,-0.714214,-1.352160,0.255227,-2.547952,0.526620,0.088701,-0.540892,-0.891786,-0.306650,-0.356961
1954,-0.182991,-1.481763,-1.455965,-0.746796,-1.231500,-1.218497,1.911915,1.267443,1.437699,-1.360032,0.288365,-2.398021,2.216956,2.379923,-0.094912,-0.805817,0.549511,-0.871570
1956,0.168639,1.264006,1.483981,1.339287,0.720917,-0.478656,0.423763,0.527056,-0.111271,-0.077323,-0.412178,1.160525,0.926492,1.268221,2.031374,1.036173,-0.341741,0.733521
1957,0.168639,1.722466,1.830741,2.269447,0.692841,-0.275653,0.134090,0.322495,-0.028442,-0.207174,-0.225799,1.200302,0.976246,1.420543,1.428699,0.886628,-0.271907,1.682467


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [41]:
model = LogisticRegression(max_iter = 1000)
model = model.fit(X_train, y_train)
preds = model.predict(X_test)

print(pct_change[test_idx].mean())

print(pct_change[y_test[preds>1].index.values].mean())



0.1270545642094541
0.3244651279020785
